### **przyjazny bot**
#### post lyrics from top 50 songs in Poland on Twitter

Import necessary libraries

In [1]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import tweepy
import random
import os
import pandas as pd
from dotenv import load_dotenv
import requests

##### Spotify API Authorization

In [2]:
load_dotenv()

auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

Download track ids from specific playlist

In [3]:
def download_tracks_ids(username, playlist_id):
    tracks_ids = []
    playlist = sp.user_playlist(username, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        tracks_ids.append(track['id'])
    return tracks_ids

In [4]:
#tracks_ids = download_tracks_ids(os.getenv("USERNAME_2"), os.getenv("PLAYLIST_ID")) 
tracks_ids = download_tracks_ids(os.getenv("USERNAME"), os.getenv("PLAYLIST_ID")) 
print(tracks_ids)

['3Nqxx5fdl3FQlrk3zarRmA', '09m29we7x9tgxyo6f3OQ87', '384lHduuDsrR4HPUwR3fG4', '4JmaOG7bfj1uNv8jEUg0Ce', '3f9Ugg030LDe0p6ospTORC', '3p6uAP2ajkobkIW8IkEpgr', '3nqQXoyQOWXiESFLlDF1hG', '4uUG5RXrOk84mYEfFvj3cK', '73vIOb4Q7YN6HeJTbscRx5', '2yxPqxHbj1L7vrCpHLtAcG', '08sMDBcjUBXWV99gZI092C', '5GdFPcbGDQqtbVO2jl6Ez5', '4nv5o5Xo4ySBaIVnXr75Xs', '1yiGGRGdHEc1F3jF5jEVmH', '561jH07mF1jHuk7KlaeF0s', '11BKm0j4eYoCPPpCONAVwA', '4N6rgD6EqLbWcl5ld3NqTP', '0zmxM4MXfisJRTQcPa1wbv', '7jtQIBanIiJOMS6RyCx6jZ', '4lPAZiTJbaQ4irMzcq0GOU', '11JXHNIH2xO5INcJQOWCjY', '53OQFPwyflA0jMyUjUSawr', '0JXXNGljqupsJaZsgSbMZV', '2KTfSSKBBfMLXwDqrtoOE5', '4h9wh7iOZ0GGn8QVp4RAOB', '5YvL3ytkusKcM5ozxD9fKN', '5NbxvSCL1xLqqWLqG6LjwE', '5XeFesFbtLpXzIVDNQP22n', '1cAAn5PKy40T1p15fpTPW8', '1Qrg8KqiBpW07V7PNxwwwL', '14154tJOTjeyeOyhnqP4QM', '2QjOHCTQ1Jl3zawyYOpxh6', '1VoUHMVxu1qG2Z9josOhWs', '2yEWVnp65tTXSrRW03IqXs', '1N8TTK1Uoy7UvQNUazfUt5', '3rb0tMq42WfggucPm0HHkA', '4Dvkj6JhhA12EX05fT7y2e', '5fjcRpwg94xQ2hrSqw6x06', '5fxFhJyveL

Draw of a track id

In [5]:
track_id = random.choice(tracks_ids)
print(track_id)

2yxPqxHbj1L7vrCpHLtAcG


Collect data (title and main artist) needed to search lyrics via the Genius API

In [6]:
def artist_genius(track_id):
    track_info = sp.track(track_id)
    artist_genius = track_info['album']['artists'][0]['name']
    return artist_genius

In [7]:
def title(track_id):
    song_info = sp.track(track_id)
    title = song_info['name']
    return title

Collect information about all artists to include in a tweet

In [8]:
def artists(track_id):
    track_info = sp.track(track_id)
    artists_list = []
    for i in range(0, len(track_info['album']['artists'])):
        artists_list.append(track_info['album']['artists'][i]['name']) 
    str_artists = ", "
    return(str_artists.join(artists_list))

In [9]:
print(artist_genius(track_id), title(track_id), artists(track_id))

Dawid Podsiadło mori Dawid Podsiadło


Get album cover for given song

In [10]:
def get_album_cover(track_id):
    track_info = sp.track(track_id = track_id)
    img_url = track_info['album']['images'][0]['url']
    img_data = requests.get(img_url).content
    with open('album_cover.jpg', 'wb') as handler:
        handler.write(img_data)

In [11]:
get_album_cover(track_id)
print(sp.track(track_id = track_id)['album']['images'][0]['url'])

https://i.scdn.co/image/ab67616d0000b2737b0be3508536803e5ff5e0bb


##### Genius API

In [12]:
genius = lyricsgenius.Genius(os.getenv("GENIUS_CLIENT_ID"))
genius.remove_section_headers = True

Download track lyrics from Genius

In [13]:
def raw_lyrics(track_id):
    lyrics = genius.search_song(title(track_id), artist_genius(track_id)).lyrics
    return lyrics

Processing the text to include it in the tweet

In [14]:
def tweet_content(lyrics, track_id):
    lyrics = lyrics.split('\n')
    for index in range(len(lyrics)):
        if lyrics[index] == "" or "[" in lyrics[index]:
            lyrics[index] = "XXX"
    lyrics = [i for i in lyrics if i != "XXX"]

    random_num = random.randrange(0, len(lyrics)-1)
    tweet = lyrics[random_num] + "\n" + lyrics[random_num+1] + "\n" + "\n" + title(track_id) + " by " + artists(track_id)
    tweet = tweet.replace("\\", "")
    return tweet

In [15]:
lyrics = raw_lyrics(track_id)
tweet_content(lyrics, track_id)

Searching for "mori" by Dawid Podsiadło...
Done.


'Czekać aż i Ty dołączysz tu\nNie wolno mi, zwlekać chwili dłużej\n\nmori by Dawid Podsiadło'

##### Twitter API

First API v1.1 authentication, only to use *media_upload* method (it's not available in API v2)

In [16]:
auth = tweepy.OAuthHandler(os.getenv("CONSUMER_KEY"), os.getenv("CONSUMER_SECRET"))
auth.set_access_token(os.getenv("ACCESS_TOKEN"), os.getenv("ACCESS_TOKEN_SECRET"))

apiv1 = tweepy.API(auth)

Upload album cover to twitter (we have to do that in order to attach it to tweet later)

In [17]:
media = apiv1.media_upload(filename = "album_cover.jpg")
media_id = media.media_id_string #media_id potem trzeba wstawić do metody create_tweet

And now API v2 authentication

In [18]:
api = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"), consumer_key= os.getenv("CONSUMER_KEY"), consumer_secret=os.getenv("CONSUMER_SECRET"), access_token=os.getenv("ACCESS_TOKEN"), access_token_secret=os.getenv("ACCESS_TOKEN_SECRET"))

Tweet lyrics of one of TOP 50 tracks in Poland

In [19]:
result = None
while result is None:
    try:
      lyrics = raw_lyrics(track_id)
      result = api.create_tweet(text=tweet_content(lyrics, track_id), media_ids = [media_id])
    except:
         pass

Searching for "mori" by Dawid Podsiadło...
Done.


#### Retweet quote tweets and reply on mention

Draw of a track id

In [20]:
track_id = random.choice(tracks_ids)
print(track_id)

11JXHNIH2xO5INcJQOWCjY


In [21]:
def get_tweet(id):
    tweet = api.get_tweet(id, expansions=['author_id'], user_fields=['username'])
    return tweet

In [22]:
def quote_tweets(query, max_results):
    tweets = api.search_recent_tweets(query=query, max_results=max_results)
    results = []
    if not tweets.data is None and len(tweets.data) > 0:
        for tweet in tweets.data:
            twt = get_tweet(tweet['id'])
            obj = {}
            obj['id'] = tweet.id
            obj['username'] = twt.includes['users'][0].username
            obj['text'] = tweet.text
            results.append(obj)
    return results

Check if whether a tweet (id) has already received a reply

In [23]:
def check(id):
    with open('last_tweet_ids.txt') as f:
        datafile = f.readlines()
    for line in datafile:
        if id in line:
            return True
    return False

Add tweet (id) to database = text file


In [24]:
def append_new_id(text_to_append):
    with open('last_tweet_ids.txt', "a+") as file_object:
        file_object.seek(0)
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        file_object.write(text_to_append)

In [25]:
QUERY = "przyjaznybot"
MAX_RESULTS = 10

quote_tweets(QUERY, MAX_RESULTS)

[{'id': 1615763990038052872,
  'username': 'Szafa9',
  'text': '@przyjaznybot draw lyrics xyz'},
 {'id': 1615763117798981652,
  'username': 'przyjaznybot',
  'text': 'Jeszcze kilka słów\nA jeśli już, to wezmę tam ze sobą bukiet róż\n\nmori by Dawid Podsiadło https://t.co/s4jzHSH7s8'},
 {'id': 1615740148905398277,
  'username': 'przyjaznybot',
  'text': 'hello @Szafa9\n\nYou might also like\nTeraz jest nasz czas! - wybiła ta godzina\n\nRarara by Miszel, Kabe https://t.co/3MRXzsLFKp'},
 {'id': 1615739685589901312,
  'username': 'Szafa9',
  'text': '@przyjaznybot draw lyrics'},
 {'id': 1615734073850314753,
  'username': 'przyjaznybot',
  'text': "Bit od CatchUp'a, płynąłeś na fali, a jesteś za burtą\nTyle jest dymu w tym jebanym studio, że mamy tu sufit, a piję pod chmurką\n\nclub2020 by club2020, Taco Hemingway, Oki https://t.co/Uf7ILJupjC"},
 {'id': 1615489554726748160,
  'username': 'przyjaznybot',
  'text': 'RT @Marhalus2: @przyjaznybot draw lyrics'},
 {'id': 1615489414863413248,
  'u

From list of dictionaries to list of specific variables

In [26]:
values_of_usernames = [dict['username'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in quote_tweets(QUERY, MAX_RESULTS)]

If someone share a tweet containing the phrase "przyjazny draw lyrics", bot create new tweet with the user's mention. <br>
If someone quotes a tweet, bot retweet it.

In [27]:
lines = []
lyrics = raw_lyrics(track_id)

get_album_cover(track_id)
media = apiv1.media_upload(filename = "album_cover.jpg")
media_id = media.media_id_string

for i in range(len(values_of_ids)):
    if check(str(values_of_ids[i])) == False:
        if values_of_usernames[i] != "przyjaznybot":
            if "draw lyrics" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content(lyrics, track_id)
                api.like(values_of_ids[i])
                api.create_tweet(text=reply, media_ids = [media_id])
                append_new_id(str(values_of_ids[i]))
            else:
                api.like(values_of_ids[i])
                api.retweet(values_of_ids[i])
                append_new_id(str(values_of_ids[i]))

Searching for "NA KRAŃCU ŚWIATA" by 2115...
Done.
